## LSTM-CNN Filter Experiment

In [1]:
%run Setup.ipynb
%run ExtraFunctions.ipynb

[i] Loaded Parameters:
 40000 35 0.2 200 
 dataset/glove/glove.twitter.27B.200d.txt
[i] Importing Modules...


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[+] Using Keras version 2.1.4
[i] Finished Importing Modules
[i] . Reading from csv file...Done!
[i] Found 30447 unique tokens.
[+] Shape of data tensor: (40000, 30)
[+] Shape of label tensor: (40000, 4)
[+] Number of entries in each category:
[+] Training:
 [ 7739. 10889. 12242.  1130.]
[+] Validation:
 [1905. 2735. 3057.  303.]
[i] Loading GloVe from: dataset/glove/glove.twitter.27B.200d.txt ...Done.
[+] Proceeding with Embedding Matrix...Completed!
Finished running setup.


Load embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [2]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### RNN Structure
[Reference](https://github.com/richliao/textClassifier), [LTSM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

In [3]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm1 = Bidirectional(LSTM(4,dropout=0.2,recurrent_dropout=0.2,return_sequences=True))(embedded_sequences)
l_cov1= Conv1D(24, 4, activation='relu')(l_lstm1)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_drop1 = Dropout(0.3)(l_pool1)
l_flat = Flatten()(l_drop1)
l_dense = Dense(16, activation='relu')(l_flat)
preds = Dense(4, activation='softmax')(l_dense)

In [4]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
lr_metric = get_lr_metric(adadelta)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [5]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoint.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [6]:
model.summary()
print("Training Progress:")
history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
                    epochs=50, batch_size=64,
                    callbacks=[lr_schedule])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 200)           6089600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 30, 8)             6560      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 27, 24)            792       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 24)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 24)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 312)               0         
__________

32000/32000 [==============================] - 22s 675us/step - loss: 0.9109 - acc: 0.6162 - val_loss: 0.9852 - val_acc: 0.5817
Epoch 49/50
32000/32000 [==============================] - 22s 675us/step - loss: 0.9113 - acc: 0.6165 - val_loss: 0.9904 - val_acc: 0.5735
Epoch 50/50
32000/32000 [==============================] - 22s 678us/step - loss: 0.9124 - acc: 0.6163 - val_loss: 0.9873 - val_acc: 0.5805


In [7]:
import pandas as pd
pd.DataFrame(history.history).to_csv("history-4.csv")
#model.save('ltsm-c-3.h5')